In [1]:
from datetime import datetime, timedelta
import requests
import pickle
from pathlib import Path

In [2]:
def get_user_reviews(review_appid, params):

    user_review_url = f'https://store.steampowered.com/appreviews/{review_appid}'
    req_user_review = requests.get(
        user_review_url,
        params=params
    )

    if req_user_review.status_code != 200:
        print(f'Fail to get response. Status code: {req_user_review.status_code}')
        return {"success": 2}
    
    try:
        user_reviews = req_user_review.json()
    except:
        return {"success": 2}

    return user_reviews

In [14]:
review_appname = 'Pixle_Puzzle'  # the game name
review_appid = 455690  # the game appid on Steam

# the params of the API
params = {
    'json': 1,
    'language': 'english',
    # 'day_range' : 9223372036854775807,
    'cursor': '*',  # set the cursor to retrieve reviews from a specific "page"
    'num_per_page': 100,
    'filter': 'all'
}

time_interval = timedelta(hours=24)  # the time interval to get the reviews
# end_time = datetime.fromtimestamp(1716718910)  # the timestamp in the return result are unix timestamp (GMT+0)
end_time = datetime(2024, 1, 1, 0, 0, 0)
# start_time = end_time - time_interval
start_time = datetime(2017, 1, 1, 0, 0, 0)

print(f"Start time: {start_time}")  # printing local timezone for logging
print(f"End time: {end_time}")
print(start_time.timestamp(), end_time.timestamp())

passed_start_time = False
passed_end_time = False

selected_reviews = []
previous_cursor = None  # To track the last cursor

while not passed_start_time or not passed_end_time:

    reviews_response = get_user_reviews(review_appid, params)

    # not success?
    if reviews_response["success"] != 1:
        print("Not a success")
        print(reviews_response)
        break

    if reviews_response["query_summary"]['num_reviews'] == 0:
        print("No reviews.")
        print(reviews_response)
        break

    for review in reviews_response["reviews"]:
        recommendation_id = review['recommendationid']

        timestamp_created = review['timestamp_created']
        timestamp_updated = review['timestamp_updated']

        # skip the comments that beyond end_time
        if not passed_end_time:
            if timestamp_created > end_time.timestamp():
                continue
            else:
                passed_end_time = True

        # exit the loop once detected a comment that before start_time
        if not passed_start_time:
            if timestamp_created < start_time.timestamp():
                passed_start_time = True
                break

        # extract the useful (to me) data
        author_steamid = review['author']['steamid']  # will automatically redirect to the profileURL if any
        playtime_forever = review['author']['playtime_forever']
        playtime_last_two_weeks = review['author']['playtime_last_two_weeks']
        playtime_at_review_minutes = review['author']['playtime_at_review']
        last_played = review['author']['last_played']

        review_language = review['language']
        review_text = review['review']
        voted_up = review['voted_up']
        votes_up = review['votes_up']
        votes_funny = review['votes_funny']
        weighted_vote_score = review['weighted_vote_score']
        steam_purchase = review['steam_purchase']
        received_for_free = review['received_for_free']
        written_during_early_access = review['written_during_early_access']

        my_review_dict = {
            'recommendationid': recommendation_id,
            'author_steamid': author_steamid,
            'playtime_at_review_minutes': playtime_at_review_minutes,
            'playtime_forever_minutes': playtime_forever,
            'playtime_last_two_weeks_minutes': playtime_last_two_weeks,
            'last_played': last_played,

            'review_language': review_language,
            'review_text': review_text,
            'timestamp_created': timestamp_created,
            'timestamp_updated': timestamp_updated,

            'voted_up': voted_up,
            'votes_up': votes_up,
            'votes_funny': votes_funny,
            'weighted_vote_score': weighted_vote_score,
            'steam_purchase': steam_purchase,
            'received_for_free': received_for_free,
            'written_during_early_access': written_during_early_access,
        }

        selected_reviews.append(my_review_dict)

    # go to the next page
    try:
        cursor = reviews_response['cursor']  # cursor field does not exist in the last page
    except KeyError:
        cursor = ''

    # Check if the cursor is the same as the previous one
    if cursor == previous_cursor:
        print("Reached the end of all comments.")
        break

    # Update the previous cursor and move to the next page
    previous_cursor = cursor
    params['cursor'] = cursor
    print('To next page. Next page cursor:', cursor)


Start time: 2017-01-01 00:00:00
End time: 2024-01-01 00:00:00
1483182000.0 1704020400.0
To next page. Next page cursor: AoIFQFZQl8AAAABwoumtBQ==
To next page. Next page cursor: AoIFQH2B0OAAAABxztikBQ==
Reached the end of all comments.


In [16]:
selected_reviews[:10]

[]

In [17]:
len(selected_reviews)

0